# Prédire les habitudes de consommation d'alcool des adolescents

# Sommaire : 

1. [Import et sélection des variables](#sect1)
2. [Transformation des variables et séparation Train / Test](#sect2)
3. [Modélisation](#sect3)
4. [Performance des modèles](#sect4)
5. [Amélioration des hyperparamètres (Grid Search)](#sect5)
6. [Autre modèles et algorithmes](#sect6)


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline

### 1. Import et sélection des vraiables<a name="sect1"></a> 

In [ ]:
student = pd.read_csv("https://raw.githubusercontent.com/udacity/machine-learning/master/projects/student_intervention/student-data.csv", sep=",").dropna()
student.rename(columns={'sex':'gender'}, inplace=True)
student['alcohol_index'] = (5*student['Dalc'] + 2*student['Walc'])/7
# Niveau de consommation d'alcool (création d'une cible
student['acl'] = student['alcohol_index'] <= 2
student['acl'] = student['acl'].map({True: 'Low', False: 'High'})

In [ ]:
student.shape

In [ ]:
student.head(3)

In [ ]:
student.dtypes

In [ ]:
# Sélection des variables / features que l'on souhaite conserver pour la prédiction
features = ['gender', 'age', 'address', 'famsize', 'Pstatus', 'Medu',
       'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout','health' ]

In [ ]:
student[features].head().T

#### Descriptif des colonnes : 

* **ADRESS** : student's home address type (binary: 'U' - urban or 'R' - rural) 
* **FAMSIZE** : Family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)
* **PSTATUS** Parent's cohabitation status (binary: 'T' - living together or 'A' - living apart)
* **MEDU** (resp. **FEDU**) : Mother's (resp. Father) education (numeric: 0 - none, 1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education, or 4 - higher education)
* **MJOB** (resp. **FJOB**): Mother's (resp. Father) job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
* **REASON** : Reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
* **GUARDIAN** : Student's guardian (nominal: 'mother', 'father' or 'other')
* **TRAVELTIME** : Home to school travel time (numeric: 1 - &lt;15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - &gt;1 hour)
* **STUDYTIME** : Weekly study time (numeric: 1 - &lt;2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - &gt;10 hours)
* **FAILURES** :           Number of past class failures (numeric: n if 1&lt;=n&lt;3, else 4)
* **SCHOOLSUP** : Extra educational support (binary: yes or no)
* **FAMSUP** : Family educational support (binary: yes or no)
* **PAID** : Extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
* **ACTIVITIES** : Extra-curricular activities (binary: yes or no) 
* **NURSERY** :  Attended nursery school (binary: yes or no)
* **HIGHER** : Wants to take higher education (binary: yes or no)
* **INTERNET** :  Internet access at home (binary: yes or no)
* **ROMANTIC** : With a romantic relationship (binary: yes or no)
* **FAMREL** : Quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
* **FREETIME** : free time after school (numeric: from 1 - very low to 5 - very high)
* **GOOUT** : Going out with friends (numeric: from 1 - very low to 5 - very high)
* **HEALTH** : Current health status (numeric: from 1 - very bad to 5 - very good)

In [ ]:
# Analyse descriptive rapide : 
student[features].describe(include='all')

Distinction des variables qualitative / quantitative et identification de la target

In [ ]:
# Qualitative :
featuresquali = ['gender','famsize','address','Pstatus','Mjob','Fjob','reason','guardian','schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic']

# Quantitative :
featuresquanti = ['age', 'Medu','Fedu',  'traveltime', 'studytime',
       'failures',  'famrel', 'freetime', 'goout','health']

# Target
target = 'acl' # consommation d'alcool

In [ ]:
student[featuresquali].describe(include='all')

In [ ]:
student = student[features + [target]]

## 2. Transformation des variables et séparation Train / Test <a name="sect2"></a> 

In [ ]:
# Avant de séparer le Train et le test on "dichotomise" les variables Quali
studentPrep = pd.get_dummies(student[features], columns=featuresquali, drop_first=True) # l'option drop first permet de supprimer une modalité comme référence (Utile dans le cas de la régression logistique)
studentPrep.head()

On transforme la variable cible en numérique afin que Scikit-learn puisse se situer dans un problème de classification binaire

In [ ]:
student['acl'] = student['acl'].map({'Low':0, 'High':1}).astype(int)

In [ ]:
# Définition des variables explicatives et de la variable Target
X = studentPrep
y = student[target]

In [ ]:
print(len(X),len(y))

Analyse de la variable cible


In [ ]:
# Taux de cible des adolescent ayant une consommation élevée d'alcool 
y.mean()

In [ ]:
y.value_counts(normalize=True)

In [ ]:
y.describe()

### Partition train /  test

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
# Application de la fonction avec tirage de 30% en test
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, test_size=0.3, random_state=42) # stratify permet de conserver la même répartition de la cible

In [ ]:
y_train.mean()

In [ ]:
y_test.mean()

#### Bonne pratique : feature scaling afin de normaliser les données 

les paramètres de standardisation sont appris sur l'échantillon d'apprentissage et réappliquer sur l'échantillon de test afin de conserver les mêmes transformation de données lors de l'apprentissage du test et de la réapplication sur de nouvelles données

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Initialisation d'on objet de normalisation 
# Normalisation : moyenne nulle et variance unitaire
scaler = StandardScaler()

In [ ]:
X_train[featuresquanti].head()

In [ ]:
# Entrainement et application de la normalisation sur les données de train 
scaler.fit(X_train[featuresquanti])
X_train[featuresquanti] = scaler.transform(X_train[featuresquanti])

In [ ]:
X_train[featuresquanti].head()

In [ ]:
X_test[featuresquanti].head()

In [ ]:
# Réapplication sur les données de test
X_test[featuresquanti] = scaler.transform(X_test[featuresquanti])

In [ ]:
X_test[featuresquanti].head()

# 3. Modélisation <a name="sect3"></a> 

## Modèle de base de la régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Initialisation de l'objet classifier
classifier_lr = LogisticRegression()#class_weight="balanced")

In [ ]:
# Apprentissage
classifier_lr.fit(X_train, y_train)

In [ ]:
# Visualisation des coefficients estimées pour chaque variable
coef=list(classifier_lr.coef_[0])
coef_df = pd.DataFrame({'Coefficients': list(coef)}, list(X_train.columns.values))
coef_df.sort_values(['Coefficients'], ascending=False)

In [ ]:
# Prédiction sur le Train et le Test
probas_train = classifier_lr.predict_proba(X_train) # pour les probas
probas_test = classifier_lr.predict_proba(X_test) # pour les probas

predict_train = classifier_lr.predict(X_train) # pour les prédictions avec cutoff = 0.5
predict_test = classifier_lr.predict(X_test)  # pour les prédictions avec cutoff = 0.5

In [ ]:
# Les Probabilités sont celles de 0 et 1 et sont complémentaires
print(classifier_lr.classes_)
probas_test[:5]

In [ ]:
# Récupération uniquement de la proba d'interêt
probas_train = probas_train[:,1]
probas_test = probas_test[:,1]

In [ ]:
# Visualisation sur un exemple : 
probas_test[0], y_test.iloc[0]

In [ ]:
X_test.iloc[0]

In [ ]:
# Vérification de la proba moyenne en fonction du paramètre class_weight="balanced"
probas_train.mean()

# 4. Métriques de performance <a name="sect4"></a> 

### On va utiliser des métriques pour évaluer le modèle
certaines métriques dépendent d'un curseur sur la proba Y=1 (threshold dependant), d'autres au contraire sont comme la logloss une quantité (threshold invariant), c'est le cas de l'AUC qui plus elle est proche de 1 meilleur sera le modèle

In [ ]:
# importation de métriques
from sklearn.metrics import classification_report,accuracy_score,f1_score,roc_auc_score,recall_score
import matplotlib.pyplot as plt

In [ ]:
# Matrice de confusion
pd.crosstab(y_train, predict_train)

In [ ]:
# Plot confusion matrice plus Visuel : 
# Récupération de la fonction dans les modules
import os
os.chdir('../')
from modules.fonctions_metrics import plot_confusion_matrix

In [ ]:
plot_confusion_matrix(y_train, predict_train, classes=["low risk","high risk"], 
                      title='Matrice de confusion échantillon Test')

Compraisons des métriques entre le train et le test

In [ ]:
print("Echantillon Train \n ---------")
print(classification_report(y_train, predict_train))

In [ ]:
print("Echantillon Test \n ---------")
print(classification_report(y_test, predict_test))

##### Tracons les courbes de ROC (compte tenu du peu de volumétrie la forme des courbes n'est pas lisse)

In [ ]:
from modules.fonctions_metrics import auc_et_roc

In [ ]:
auc_et_roc(y_train,probas_train)
auc_et_roc(y_test,probas_test)

##### Tracons les courbes de Gain et Lift (compte tenu du peu de volumétrie la forme des courbes n'est pas lisse)

In [ ]:
from modules.fonctions_metrics import CAP_table, lift 

In [ ]:
table_lift = CAP_t.able(pd.Series(probas_train,index=y_train.index), y_train,stepsize = 1, n=100)
table_lift

In [ ]:
plt.plot(table_lift['% positifs cumulés sur le total des positifs'], label="Mon Modèle")
plt.plot([0, 100], [0, 100], c = 'r', linestyle = '--', label = 'Modèle aléatoire')
plt.plot([0,  np.round(y_train.sum() / len(y_train)*100,0), 100], [0, 100, 100], c = 'g', linestyle = '-', label = 'Modèle parfait')
plt.legend()
plt.xlabel('% de scorés')
plt.ylabel('% de scorés à raison')

In [ ]:
plt.plot(table_lift['Lift'])
plt.xlabel('% de scorés')
plt.ylabel('Lift')

# 5. Amélioration des hyperparamètres (Grid Search) <a name="sect5"></a> 

## Modèle de régression logistique pénalisée

Une régression pénalisée de type ridge (L2) permet de contraindre l'espace des coef estimés pour ne pas qu'ils prennent des valeurs contradictoires et très élevées,
Si la régression est de type lasso (L1) alors certains coefficients vont être annulés.
Le paramètre C contrôle cela : 
C = Inverse of regularization strength; must be a positive float = smaller values specify stronger regularization.

**A noter que dans certaines classes, le paramètre est 1/C**

*NB : Dans Sklearn le coefficient C est égale à l'inverse du poids de régularization*


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# On définit la liste des hyper paramètres de l'on souhaite tester
param = [{ "C": [0.01,0.025,0.05,0.1],"class_weight": ["balanced", None] }]

# On initialise un objet classifier LR sur lequel nous allons tester toutes les itérations possibles
clf_lr=LogisticRegression(penalty='l1', solver="liblinear")

# On initialise un objet gridsearch grâce à la liste d'arguments et au classifier 
# on lui donne une métrique à maximiser pour identifier le meilleur modèle
# on spécifie sur combien d'échantillon de cross validation se fera le calcul des métriques (cv)  
modelCV= GridSearchCV(clf_lr, param, cv = 4, n_jobs = -1, scoring="recall")


In [ ]:
# On lance l'entraînement de tous les modèles
modelCV = modelCV.fit(X_train, y_train)

In [ ]:
# Résultats détaillés :
pd.DataFrame(modelCV.cv_results_)

In [ ]:
# Quel sont les meilleurs paramètres ? 
modelCV.best_params_

In [ ]:
# Meilleur estimator
modelCV.best_estimator_

In [ ]:
# Visualisation des coefficients estimées pour chaque variable
coef=list(modelCV.best_estimator_.coef_[0])
coef_df = pd.DataFrame({'Coefficients': list(coef)}, list(X_train.columns.values))

coef_df.sort_values(['Coefficients'], ascending=False)

In [ ]:
# L'argument refit=True permet de réentrainer directement le meilleur estimator à la fin de l'entrainement du Grid Search
# on a donc pas besoin de ré-entraîner un modèle mais on peut directement utiliser l'objet modelCV
# Dans le cas où le best_estimator_ ne nous convient pas il faudrait entraîner un nouveau modèle avec les paramètres voulus

predict_train = modelCV.predict(X_train)
predict_test = modelCV.predict(X_test)
probas_train = modelCV.predict_proba(X_train)[:,1]
probas_test = modelCV.predict_proba(X_test)[:,1]


#### Evaluation des performances

In [ ]:
print("Echantillon Train \n ---------")
print(classification_report(y_train, predict_train))

In [ ]:
print("Echantillon Test \n ---------")
print(classification_report(y_test, predict_test))

In [ ]:
auc_et_roc(y_train,probas_train)
auc_et_roc(y_test,probas_test)

# 6. Autre modèles et algorithmes <a name="sect6" ></a>

### Arbre CART

In [ ]:
from sklearn.tree import DecisionTreeClassifier
modele_arbre=DecisionTreeClassifier(random_state = 42, max_depth = 3, min_samples_leaf = 30)
modele_arbre.fit(X_train, y_train)
predict_train = modele_arbre.predict(X_train)
predict_test = modele_arbre.predict(X_test)
probas_train = modele_arbre.predict_proba(X_train)[:,1]
probas_test = modele_arbre.predict_proba(X_test)[:,1]
print("Echantillon Train \n ---------")
print(classification_report(y_train, predict_train))

print("Echantillon Test \n ---------")
print(classification_report(y_test, predict_test))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
modele_arbre=DecisionTreeClassifier(random_state = 42, max_depth = 3, min_samples_leaf = 30)

In [ ]:
modele_arbre.fit(X_train, y_train)

In [ ]:
predict_train = modele_arbre.predict(X_train)
predict_test = modele_arbre.predict(X_test)
probas_train = modele_arbre.predict_proba(X_train)[:,1]
probas_test = modele_arbre.predict_proba(X_test)[:,1]


In [ ]:
print("Echantillon Train \n ---------")
print(classification_report(y_train, predict_train))

print("Echantillon Test \n ---------")
print(classification_report(y_test, predict_test))

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(20, 20))
tree.plot_tree(modele_arbre,class_names=True, max_depth=4,proportion=True, fontsize=10,filled=True,feature_names=X_train.columns) 

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 300, max_depth = 3,min_samples_leaf = 30, random_state = 42, class_weight="balanced" )    
rf.fit(X_train, y_train)
predict_train = rf.predict(X_train)
predict_test = rf.predict(X_test)
probas_train = rf.predict_proba(X_train)[:,1]
probas_test = rf.predict_proba(X_test)[:,1]

In [ ]:
print("Echantillon Train \n ---------")
print(classification_report(y_train, predict_train))

print("Echantillon Test \n ---------")
print(classification_report(y_test, predict_test))

In [ ]:
pd.DataFrame([X_train.columns,rf.feature_importances_]).T.sort_values([1],ascending=False)

In [ ]:
auc_et_roc(y_train,probas_train)
auc_et_roc(y_test,probas_test)

### GBM (Gradient Boosting Machine) 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbm =GradientBoostingClassifier(learning_rate=0.01,
                           n_estimators=400, max_depth=2, random_state=42,subsample=0.9 )
gbm.fit(X_train, y_train)
predict_train = gbm.predict(X_train)
predict_test = gbm.predict(X_test)
probas_train = gbm.predict_proba(X_train)[:,1]
probas_test = gbm.predict_proba(X_test)[:,1]

In [ ]:
print("Echantillon Train \n ---------")
print(classification_report(y_train, predict_train))

print("Echantillon Test \n ---------")
print(classification_report(y_test, predict_test))

In [ ]:
pd.DataFrame([X_train.columns,gbm.feature_importances_]).T.sort_values([1],ascending=False)